In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, when, sum

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("FootballTournament").getOrCreate()

# Define Schema for Teams Table
teams_schema = StructType([
    StructField("team_id", IntegerType(), False),
    StructField("team_name", StringType(), False)
])

# Define Schema for Matches Table
matches_schema = StructType([
    StructField("match_id", IntegerType(), False),
    StructField("host_team", IntegerType(), False),
    StructField("guest_team", IntegerType(), False),
    StructField("host_goals", IntegerType(), False),
    StructField("guest_goals", IntegerType(), False)
])

# Hardcoded Teams Data
teams_data = [
    (10, "Leetcode FC"),
    (20, "NewYork FC"),
    (30, "Atlanta FC"),
    (40, "Chicago FC"),
    (50, "Toronto FC")
]

# Hardcoded Matches Data
matches_data = [
    (1, 10, 20, 3, 0),
    (2, 30, 10, 2, 2),
    (3, 10, 50, 5, 1),
    (4, 20, 30, 1, 0),
    (5, 50, 30, 1, 0)
]

# Create DataFrames
teams_df = spark.createDataFrame(teams_data, schema=teams_schema)
matches_df = spark.createDataFrame(matches_data, schema=matches_schema)

# Display DataFrames
print("Teams Table:")
teams_df.show()

print("Matches Table:")
matches_df.show()

25/02/16 23:08:43 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/16 23:08:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/16 23:08:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Teams Table:


+-------+-----------+
|team_id|  team_name|
+-------+-----------+
|     10|Leetcode FC|
|     20| NewYork FC|
|     30| Atlanta FC|
|     40| Chicago FC|
|     50| Toronto FC|
+-------+-----------+

Matches Table:
+--------+---------+----------+----------+-----------+
|match_id|host_team|guest_team|host_goals|guest_goals|
+--------+---------+----------+----------+-----------+
|       1|       10|        20|         3|          0|
|       2|       30|        10|         2|          2|
|       3|       10|        50|         5|          1|
|       4|       20|        30|         1|          0|
|       5|       50|        30|         1|          0|
+--------+---------+----------+----------+-----------+



In [2]:
# Create views for the dataframes
teams_df.createOrReplaceTempView("teams_view")
matches_df.createOrReplaceTempView("matches_view")

In [3]:
spark.sql("""
    WITH match_pts_calculated AS (
            SELECT *,
                CASE
                    WHEN host_goals > guest_goals THEN 'host win'
                    WHEN host_goals < guest_goals THEN 'guest win'
                    ELSE 'draw'
                END as result,
                CASE
                    WHEN host_goals > guest_goals THEN 3
                    WHEN host_goals < guest_goals THEN 0
                    ELSE 1
                END as host_pts,
                CASE
                    WHEN guest_goals > host_goals THEN 3
                    WHEN guest_goals < host_goals THEN 0
                    ELSE 1
                END as guest_pts
            FROM matches_view
          ),
    
    pts_calculated AS (
                SELECT host_team as team,
                        SUM(host_pts) as pts
                FROM match_pts_calculated
                GROUP BY host_team
                UNION
                SELECT guest_team as team,
                        SUM(guest_pts) as pts
                FROM match_pts_calculated
                GROUP BY guest_team
          )

    SELECT team as team_id, SUM(pts) as num_pts
    FROM pts_calculated
    GROUP BY team
    ORDER BY num_pts DESC
""").show()

+-------+-------+
|team_id|num_pts|
+-------+-------+
|     10|      7|
|     20|      3|
|     50|      3|
|     30|      1|
+-------+-------+



In [1]:
spark.stop()

NameError: name 'spark' is not defined